# Fine Tuning Roberta for Text classification





### Introduction

In this tutorial I will be fine tuning a roberta model for the **Text classification** problem.

### What is a text classification

In the e-cigarette product example, a text classifcation is to classify whether the current text should be about against/support e-cigarette or neural. In this setting, then the classification will be three categories. Based on your definition, you can classify the text into the predefined labels.

#### Flow of the notebook

The notebook will be divided into seperate sections to provide a organized walk through for the process used. This process can be modified for individual use cases. The sections are:

1. [Importing Python Libraries and preparing the environment](#section01)
2. [Importing and Pre-Processing the domain data](#section02)
3. [Preparing the Dataset and Dataloader](#section03)
4. [Creating the Neural Network for Fine Tuning](#section04)
5. [Fine Tuning the Model](#section05)
6. [Validating the Model Performance](#section06)
7. [Saving the model and artifacts for Inference in Future](#section07)

#### Technical Details

This script leverages on multiple tools designed by other teams. Details of the tools used below. Please ensure that these elements are present in your setup to successfully implement this script.

 - Data:
	 - I will be using the dataset available at [Kaggle Competition](https://www.kaggle.com/c/movie-review-sentiment-analysis-kernels-only)
	 - I will be referring only to the first csv file from the data dump: `train.tsv`


 - Hardware Requirements:
	 - Python 3.6 and above
	 - Pytorch, Transformers and All the stock Python ML Libraries
	 - GPU enabled setup

<a id='section01'></a>
### Importing Python Libraries and preparing the environment

At this step we will be importing the libraries and modules needed to run our script. Libraries are:
* Pandas
* Pytorch
* Pytorch Utils for Dataset and Dataloader
* Transformers
* tqdm
* sklearn
* Robert Model and Tokenizer

Followed by that we will preapre the device for CUDA execeution. This configuration is needed if you want to leverage on onboard GPU.

In [1]:
!pip install transformers imblearn fuzzywuzzy

In [2]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
from imblearn.under_sampling import RandomUnderSampler
import logging
logging.basicConfig(level=logging.ERROR)

In [3]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [4]:
attitude_df = pd.read_csv('augmented_train_data.csv',
                 names=["attitude", "text"],
                 encoding="utf-8", encoding_errors="replace")
user_df = pd.read_csv('random_2300.csv')

In [5]:
attitude_df.shape
user_df.shape

(2300, 55)

In [6]:
attitude_df.head()

attitude                                               text
0  positive  380k views, 13.3k likes?!?! All jokes aside th...
1  positive  @Fa_ali00 Wife is home in texas while im in LA...
2   neutral  On a study date at 5pm and this nigga ordered ...
3   neutral       i kind of miss working at the hookah lounge,
4  negative  Sucking for dear life on the hookah tip kinda ...

In [7]:
user_df.head()

ID        ID_str  \
0  1.413381e+18  1.413381e+18   
1  1.380588e+18  1.380588e+18   
2  1.485012e+18  1.485012e+18   
3  1.471325e+18  1.471325e+18   
4  1.463011e+18  1.463011e+18   

                                                Text  attitude user or not  \
0                   I just like shisha on Thursday..  positive        user   
1  levels gonna be so mf lit tomorrow. #PatronPal...     other    not user   
2  How tf do y’all smoke hookah 😩 I be bout ready...  negative    not user   
3  @_djchevy Getting little homies ready! Can we ...     other        user   
4  Hookah dont do shit but make me feel light hea...  negative        user   

   Unnamed: 5  Unnamed: 6  Unnamed: 7  Unnamed: 8  Unnamed: 9  ...  \
0         NaN         NaN         NaN         NaN         NaN  ...   
1         NaN         NaN         NaN         NaN         NaN  ...   
2         NaN         NaN         NaN         NaN         NaN  ...   
3         NaN         NaN         NaN         NaN         NaN  ...   
4         NaN         NaN         NaN         NaN         NaN  ...   

   Unnamed: 45  Unnamed: 46  Unnamed: 47  Unnamed: 48  Unnamed: 49  \
0          NaN          NaN          NaN          NaN          NaN   
1          NaN          NaN          NaN          NaN          NaN   
2          NaN          NaN          NaN          NaN          NaN   
3          NaN          NaN          NaN          NaN          NaN   
4          NaN          NaN          NaN          NaN          NaN   

   Unnamed: 50  Unnamed: 51  Unnamed: 52  Unnamed: 53  Unnamed: 54  
0          NaN          NaN          NaN          NaN          NaN  
1          NaN          NaN          NaN          NaN          NaN  
2          NaN          NaN          NaN          NaN          NaN  
3          NaN          NaN          NaN          NaN          NaN  
4          NaN          NaN          NaN          NaN          NaN  

[5 rows x 55 columns]

In [8]:
attitude_df['attitude'] = attitude_df['attitude'].str.strip() 
user_df['user or not'] = user_df['user or not'].str.replace('uesr', 'user').str.replace('not$', 'not user').str.strip()

<ipython-input-8-b2b335277417>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  user_df['user or not'] = user_df['user or not'].str.replace('uesr', 'user').str.replace('not$', 'not user').str.strip()


In [9]:
mapping_attitude = {
    'positive': 0,
    'negative': 1,
    'neutral': 2,
}

mapping_user = {
    'user': 0,
    'not user': 1
}

attitude_df['attitude'] = attitude_df['attitude'].map(mapping_attitude)
user_df['user or not'] = user_df['user or not'].map(mapping_user)

user_df = user_df.rename(columns={'user or not': 'user_or_not'})

In [10]:
attitude_df.head()

attitude                                               text
0         0  380k views, 13.3k likes?!?! All jokes aside th...
1         0  @Fa_ali00 Wife is home in texas while im in LA...
2         2  On a study date at 5pm and this nigga ordered ...
3         2       i kind of miss working at the hookah lounge,
4         1  Sucking for dear life on the hookah tip kinda ...

In [11]:
user_df.head()

ID        ID_str  \
0  1.413381e+18  1.413381e+18   
1  1.380588e+18  1.380588e+18   
2  1.485012e+18  1.485012e+18   
3  1.471325e+18  1.471325e+18   
4  1.463011e+18  1.463011e+18   

                                                Text  attitude  user_or_not  \
0                   I just like shisha on Thursday..  positive            0   
1  levels gonna be so mf lit tomorrow. #PatronPal...     other            1   
2  How tf do y’all smoke hookah 😩 I be bout ready...  negative            1   
3  @_djchevy Getting little homies ready! Can we ...     other            0   
4  Hookah dont do shit but make me feel light hea...  negative            0   

   Unnamed: 5  Unnamed: 6  Unnamed: 7  Unnamed: 8  Unnamed: 9  ...  \
0         NaN         NaN         NaN         NaN         NaN  ...   
1         NaN         NaN         NaN         NaN         NaN  ...   
2         NaN         NaN         NaN         NaN         NaN  ...   
3         NaN         NaN         NaN         NaN         NaN  ...   
4         NaN         NaN         NaN         NaN         NaN  ...   

   Unnamed: 45  Unnamed: 46  Unnamed: 47  Unnamed: 48  Unnamed: 49  \
0          NaN          NaN          NaN          NaN          NaN   
1          NaN          NaN          NaN          NaN          NaN   
2          NaN          NaN          NaN          NaN          NaN   
3          NaN          NaN          NaN          NaN          NaN   
4          NaN          NaN          NaN          NaN          NaN   

   Unnamed: 50  Unnamed: 51  Unnamed: 52  Unnamed: 53  Unnamed: 54  
0          NaN          NaN          NaN          NaN          NaN  
1          NaN          NaN          NaN          NaN          NaN  
2          NaN          NaN          NaN          NaN          NaN  
3          NaN          NaN          NaN          NaN          NaN  
4          NaN          NaN          NaN          NaN          NaN  

[5 rows x 55 columns]

In [12]:
print(attitude_df.isna().sum())


attitude    0
text        0
dtype: int64


In [13]:
print(user_df.isna().sum())

ID                0
ID_str            0
Text              0
attitude          0
user_or_not       0
Unnamed: 5     2300
Unnamed: 6     2300
Unnamed: 7     2300
Unnamed: 8     2300
Unnamed: 9     2300
Unnamed: 10    2300
Unnamed: 11    2300
Unnamed: 12    2300
Unnamed: 13    2300
Unnamed: 14    2300
Unnamed: 15    2300
Unnamed: 16    2300
Unnamed: 17    2300
Unnamed: 18    2300
Unnamed: 19    2300
Unnamed: 20    2300
Unnamed: 21    2300
Unnamed: 22    2300
Unnamed: 23    2300
Unnamed: 24    2300
Unnamed: 25    2300
Unnamed: 26    2300
Unnamed: 27    2300
Unnamed: 28    2300
Unnamed: 29    2300
Unnamed: 30    2300
Unnamed: 31    2300
Unnamed: 32    2300
Unnamed: 33    2300
Unnamed: 34    2300
Unnamed: 35    2300
Unnamed: 36    2300
Unnamed: 37    2300
Unnamed: 38    2300
Unnamed: 39    2300
Unnamed: 40    2300
Unnamed: 41    2300
Unnamed: 42    2300
Unnamed: 43    2300
Unnamed: 44    2300
Unnamed: 45    2300
Unnamed: 46    2300
Unnamed: 47    2300
Unnamed: 48    2300
Unnamed: 49    2300


<a id='section03'></a>
### Preparing the Dataset and Dataloader

I will start with defining few key variables that will be used later during the training/fine tuning stage.
Followed by creation of Dataset class - This defines how the text is pre-processed before sending it to the neural network. I will also define the Dataloader that will feed  the data in batches to the neural network for suitable training and processing.
Dataset and Dataloader are constructs of the PyTorch library for defining and controlling the data pre-processing and its passage to neural network. For further reading into Dataset and Dataloader read the [docs at PyTorch](https://pytorch.org/docs/stable/data.html)

#### *TextData* Dataset Class
- This class is defined to accept the Dataframe as input and generate tokenized output that is used by the Roberta model for training.
- I am using the Roberta tokenizer to tokenize the data in the `TITLE` column of the dataframe.
- The tokenizer uses the `encode_plus` method to perform tokenization and generate the necessary outputs, namely: `ids`, `attention_mask`
- To read further into the tokenizer, [refer to this document](https://huggingface.co/transformers/model_doc/roberta.html#robertatokenizer)
- `target` is the encoded category on the news headline.
- The *SentimentData* class is used to create 2 datasets, for training and for validation.
- *Training Dataset* is used to fine tune the model: **80% of the original data**
- *Validation Dataset* is used to evaluate the performance of the model. The model has not seen this data during training.

#### Dataloader
- Dataloader is used to for creating training and validation dataloader that load data to the neural network in a defined manner. This is needed because all the data from the dataset cannot be loaded to the memory at once, hence the amount of dataloaded to the memory and then passed to the neural network needs to be controlled.
- This control is achieved using the parameters such as `batch_size` and `max_len`.
- Training and Validation dataloaders are used in the training and validation part of the flow respectively

In [14]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
# EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [15]:

import re




class attitudeTextData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.attitude = self.data.attitude
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def text_preprocessing(self, text):
      """
      - Remove entity mentions (eg. '@united')
      - Correct errors (eg. '&amp;' to '&')
      @param    text (str): a string to be processed.
      @return   text (Str): the processed string.
      """
      # Remove '@name'
      text = re.sub(r'(@.*?)[\s]', ' ', text)

      # Replace '&amp;' with '&'
      text = re.sub(r'&amp;', '&', text)

      # Remove trailing whitespace
      text = re.sub(r'\s+', ' ', text).strip()

      return text

    def __getitem__(self, index):
        text = self.text_preprocessing(str(self.text[index]))
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'attitude': torch.tensor(self.attitude[index], dtype=torch.float),
        }

In [16]:

import re




class userTextData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.Text
        self.user_or_not = self.data.user_or_not
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def text_preprocessing(self, text):
      """
      - Remove entity mentions (eg. '@united')
      - Correct errors (eg. '&amp;' to '&')
      @param    text (str): a string to be processed.
      @return   text (Str): the processed string.
      """
      # Remove '@name'
      text = re.sub(r'(@.*?)[\s]', ' ', text)

      # Replace '&amp;' with '&'
      text = re.sub(r'&amp;', '&', text)

      # Remove trailing whitespace
      text = re.sub(r'\s+', ' ', text).strip()

      return text

    def __getitem__(self, index):
        text = self.text_preprocessing(str(self.text[index]))
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'user_or_not' : torch.tensor(self.user_or_not[index], dtype=torch.float)
        }

In [ ]:
test_data = pd.read_excel('test_data.xlsx')
test_data = test_data[['Text', 'attitude']]
test_data.columns = ['text', 'attitude']

In [ ]:
undersample_strategy = {0: 2500}
undersampler = RandomUnderSampler(sampling_strategy=undersample_strategy,random_state=42)

X = attitude_df.drop('attitude', axis=1)
y = attitude_df['attitude']

X_resampled, y_resampled = undersampler.fit_resample(X, y)

attitude_df = pd.DataFrame(X_resampled, columns=["text"])
attitude_df['attitude'] = y_resampled

train_data=attitude_df
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(attitude_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = attitudeTextData(train_data, tokenizer, MAX_LEN)
testing_set = attitudeTextData(test_data, tokenizer, MAX_LEN)

In [18]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

<a id='section04'></a>
### Creating the Neural Network for Fine Tuning

#### Neural Network
 - We will be creating a neural network with the `RobertaClass`.
 - This network will have the Roberta Language model followed by a `dropout` and finally a `Linear` layer to obtain the final outputs.
 - The data will be fed to the Roberta Language model as defined in the dataset.
 - Final layer outputs is what will be compared to the `Sentiment category` to determine the accuracy of models prediction.
 - We will initiate an instance of the network called `model`. This instance will be used for training and then to save the final trained model for future inference.

#### Loss Function and Optimizer
 - `Loss Function` and `Optimizer` and defined in the next cell.
 - The `Loss Function` is used the calculate the difference in the output created by the model and the actual output.
 - `Optimizer` is used to update the weights of the neural network to improve its performance.

In [19]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 4)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [20]:
model = RobertaClass()
model.to(device)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

<a id='section05'></a>
### Fine Tuning the Model

After all the effort of loading and preparing the data and datasets, creating the model and defining its loss and optimizer. This is probably the easier steps in the process.

Here we define a training function that trains the model on the training dataset created above, specified number of times (EPOCH), An epoch defines how many times the complete data will be passed through the network.

Following events happen in this function to fine tune the neural network:
- The dataloader passes data to the model based on the batch size.
- Subsequent output from the model and the actual category are compared to calculate the loss.
- Loss value is used to optimize the weights of the neurons in the network.
- After every 5000 steps the loss value is printed in the console.

As you can see just in 1 epoch by the final step the model was working with a loss of 0.8141926634122427.

In [21]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [22]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [23]:
from sklearn.metrics import f1_score

def calculate_f1score(preds, targets):
  score = f1_score(targets, preds, average='macro')
  return score


In [24]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train_attitude(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0

    train_loss_list = []

    model.train()


    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['attitude'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_function(outputs, targets)

        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return epoch_loss

In [ ]:
import matplotlib.pyplot as plt

train_loss_list = []

EPOCHS = 15
for epoch in range(EPOCHS):
    epoch_loss = train_attitude(epoch)
    train_loss_list.append(epoch_loss)

plt.figure()
plt.plot(train_loss_list, label='training loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training Loss per 5000 steps: 1.457353115081787
Training Accuracy per 5000 steps: 25.0


1691it [10:39,  2.64it/s]


The Total Accuracy for Epoch 0: 52.82377291543465
Training Loss Epoch: 0.9692164182239986
Training Accuracy Epoch: 52.82377291543465


1it [00:00,  2.67it/s]

...

1691it [10:42,  2.63it/s]


The Total Accuracy for Epoch 2: 84.49142519219397
Training Loss Epoch: 0.41353973660492493
Training Accuracy Epoch: 84.49142519219397


1it [00:00,  2.73it/s]

Training Loss per 5000 steps: 0.020110176876187325
Training Accuracy per 5000 steps: 100.0


951it [06:01,  2.64it/s]

<a id='section06'></a>
### Validating the Model

During the validation stage we pass the unseen data(Testing Dataset) to the model. This step determines how good the model performs on the unseen data.

This unseen data is the 20% of dataset which was seperated during the Dataset creation stage.
During the validation stage the weights of the model are not updated. Only the final output is compared to the actual value. This comparison is then used to calcuate the accuracy of the model.

In [ ]:
def valid_attitude(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0

    predictions_list = []
    targets_list = []

    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['attitude'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)

            predictions_list.extend(big_idx.cpu().numpy())
            targets_list.extend(targets.cpu().numpy())

            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)

            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    f1_score = calculate_f1score(predictions_list, targets_list)

    return epoch_accu, f1_score

In [ ]:
acc_attitude,f1_score_attitude = valid_attitude(model, testing_loader)
print("Accuracy on attitude of test data = %0.2f%%" % acc_attitude)
print("F1-score on attitude of test data = %0.2f%%" % f1_score_attitude)

<a id='section07'></a>
### Saving the Trained Model Artifacts for inference

This is the final step in the process of fine tuning the model.

The model and its vocabulary are saved locally. These files are then used in the future to make inference on new inputs of news headlines.

In [ ]:
output_model_file = 'pytorch_roberta_textclassify_attitude.bin'


model_to_save = model
torch.save(model_to_save, output_model_file)

print('All files saved')


### Preparing the Dataset and Dataloader

In [ ]:
train_size = 0.8
train_data=user_df.sample(frac=train_size,random_state=200)
test_data=user_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(user_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = userTextData(train_data, tokenizer, MAX_LEN)
testing_set = userTextData(test_data, tokenizer, MAX_LEN)

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

### Fine Tuning the Model

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train_user(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0

    train_loss_list = []

    model.train()


    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['user_or_not'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_function(outputs, targets)

        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return epoch_loss

In [ ]:
import matplotlib.pyplot as plt

train_loss_list = []

EPOCHS = 5
for epoch in range(EPOCHS):
    epoch_loss = train_attitude(epoch)
    train_loss_list.append(epoch_loss)

plt.figure()
plt.plot(train_loss_list, label='training loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

### Validating the Model

In [ ]:
def valid_user(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0

    predictions_list = []
    targets_list = []

    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['user_or_not'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)

            predictions_list.extend(big_idx.cpu().numpy())
            targets_list.extend(targets.cpu().numpy())

            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)

            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    f1_score = calculate_f1score(predictions_list, targets_list)

    return epoch_accu, f1_score

In [ ]:
acc_user, f1_score_user = valid_user(model, testing_loader)
print("Accuracy on user or not of test data = %0.2f%%" % acc_user)
print("F1-score on user or not of test data = %0.2f%%" % f1_score_user)

### Saving the Trained Model Artifacts for inference

In [ ]:
output_model_file = 'pytorch_roberta_textclassify_user.bin'


model_to_save = model
torch.save(model_to_save, output_model_file)

print('All files saved')

# Predict

In [ ]:
data_set = pd.read_csv('random_data_non.csv')
data_set.head()

In [ ]:
class Pred_TextData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.Text
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def text_preprocessing(self, text):
      """
      - Remove entity mentions (eg. '@united')
      - Correct errors (eg. '&amp;' to '&')
      @param    text (str): a string to be processed.
      @return   text (Str): the processed string.
      """
      # Remove '@name'
      text = re.sub(r'(@.*?)[\s]', ' ', text)

      # Replace '&amp;' with '&'
      text = re.sub(r'&amp;', '&', text)

      # Remove trailing whitespace
      text = re.sub(r'\s+', ' ', text).strip()

      return text

    def __getitem__(self, index):
        text = self.text_preprocessing(str(self.text[index]))
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
        }

In [ ]:
pred_data = Pred_TextData(data_set, tokenizer, MAX_LEN)

In [ ]:
pred_params = {'batch_size': 4,
                'shuffle': False,
                'num_workers': 0
                }

pred_loader = DataLoader(pred_data, **pred_params)

In [ ]:
# reload the model

model_attitude = torch.load('pytorch_roberta_textclassify_attitude.bin')
model_user = torch.load('pytorch_roberta_textclassify_user.bin')

In [ ]:
def prediction(model, pred_loader):
    predictions = []
    model.eval()
    with torch.no_grad():
        for _, data in tqdm(enumerate(pred_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            big_val, big_idx = torch.max(outputs.data, dim=1)

            predictions.extend(big_idx.cpu().numpy())

    return predictions

In [ ]:
user_prediction = prediction(model_user, pred_loader)

In [ ]:
data_set['user_or_not'] = user_prediction
data_set.head(10)

In [ ]:
attitude_prediction = prediction(model_attitude, pred_loader)

In [ ]:
data_set['attitude'] = attitude_prediction
data_set.head()

In [ ]:
inverse_mapping = {v: k for k, v in mapping.items()}
inverse_mapping_user = {v: k for k, v in mapping_user.items()}


In [ ]:
print(inverse_mapping)

In [ ]:
data_set['attitude'] = data_set['attitude'].map(inverse_mapping)
data_set['user_or_not'] = data_set['user_or_not'].map(inverse_mapping_user)
train_set['attitude'] = train_set['attitude'].map(inverse_mapping)
train_set['user_or_not'] = train_set['user_or_not'].map(inverse_mapping_user)

In [ ]:
data_set.head()

In [ ]:
train_set.head()

In [ ]:
data_set = data_set.drop('Timestamp', axis = 1)

In [ ]:
data_set = data_set[['ID', 'ID_str', 'Text','attitude','user_or_not']]

In [ ]:
data_set.head()

In [ ]:
data = pd.concat([data_set, train_set],ignore_index = True)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data_set.to_csv('prediction_label_result.csv',index=False, header=True, encoding='utf-8-sig')
train_set.to_csv('human_label_result.csv',index=False, header=True, encoding='utf-8-sig')
data.to_csv('total_result.csv',index=False, header = True, encoding='utf-8-sig')